## INCLUDES:

# LOGISTIC REGRESSION

# DECISION TREE CLASSIFIER

# RANDOM FOREST CLASSIFIER

# GRADIENT BOOSTING CLASSIFIER

In [3]:
import pandas as pd
import numpy as np
from random import *
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score

import my_pickle as mp
import my_features as mf
import my_resample as ms
# from my_resample import div_count_pos_neg, undersample, oversample
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve, auc
from scipy import interp


from sklearn.linear_model import LogisticRegression


from sklearn.ensemble.partial_dependence import plot_partial_dependence
from sklearn.ensemble.partial_dependence import partial_dependence

In [4]:
df = mp.unjson_it('data_user_active')

In [5]:
y = df.active
TF = {True:1, False:0}
y = y.map(TF)

In [6]:
X = df.drop(['active'], axis=1)

In [7]:
X = X.fillna(X.mean(axis=0))

In [8]:
mp.json_it(X,'save_the_filled_array')

In [9]:
X = mp.unjson_it('save_the_filled_array')

# Clean Data

In [10]:
X.amenities = X.amenities.apply(lambda x: len(x) if isinstance(x,list) else 0)
X.hobbies = X.hobbies.apply(lambda x: len(x) if isinstance(x,list) else 0)
X.neighborhoods = X.neighborhoods.apply(lambda x: len(x) if isinstance(x,list) else 0)
X.location = X.location.apply(lambda x: 1 if isinstance(x,list) else 0)
X['len_about'] = X.about.apply(len)
X['has_about'] = X['len_about'].apply(lambda x: x>0)
X['has_about']= X['has_about'].map(TF)
X.college = X.college.apply(lambda x: 1 if isinstance(x,str) else 0)
X.hometownCity = X.hometownCity.apply(lambda x: 1 if isinstance(x,str) else 0)
X.hometownCountry = X.hometownCountry.apply(lambda x: 1 if isinstance(x,str) else 0)
X.hometownState = X.hometownState.apply(lambda x: 1 if isinstance(x,str) else 0)
X.work = X.work.apply(lambda x: 1 if isinstance(x,str) else 0)

TF = {True:1, False:0}
TF_col = ['facebookId','has_about','has_room','linkedinId','picture','has_about']
for col in TF_col:
    X[col]= X[col].map(TF)
to_drop = ['type','uid','about','metro','picture','facebookId','linkedinId']
X = X.drop(to_drop, axis=1)

# PREPARE DATA

same train test split, resample, and scaling for everything

In [11]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X.as_matrix(), y.as_matrix(), random_state=17)

# resample
X_train, y_train = ms.oversample(X_train, y_train, .5)

# scale data
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# make fake data
pred_all_0 = [0]*len(y_test)
pred_all_1 = [1]*len(y_test)
pred_50_50 = np.random.choice([0,1], size=len(y_test))
pred_90_10 = np.random.choice([0,1], size=len(y_test), p=[.9,.1])

# MAIN FN

In [13]:
# def do_stuff(model):
#     X_train, X_test, y_train, y_test = train_test_split(X_full.as_matrix(), y.as_matrix(), random_state=17)

#     X_train, y_train = ms.oversample(X_train, y_train, .5)

#     model.fit(X_train, y_train)
#     y_pred = model.predict(X_test)

#     print("\nMETRICS")
#     print("Model recall: {}".format(recall_score(y_test, y_pred)))
#     print("Model precision: {}".format(precision_score(y_test, y_pred)))
#     print("Model accuracy: {}".format(model.score(X_test, y_test)))

#     pred_all_0 = [0]*len(y_test)
#     pred_all_1 = [1]*len(y_test)
#     pred_50_50 = np.random.choice([0,1], size=len(y_test))
#     pred_90_10 = np.random.choice([0,1], size=len(y_test), p=[.9,.1])

#     print ("\nCONFUSION MATRIX")
#     print (confusion_matrix(y_test, y_pred))
#     print ("\nkey:")
#     print (" TN   FP ")
#     print (" FN   TP ")

#     print("\nRECALL AND ACCURACY FOR DIFFERNET MODELS")
#     print("recall     \t precision   \tmodel")
#     print(recall_score(y_test, y_pred), '\t',precision_score(y_test, y_pred), "my model")
#     print(recall_score(y_test, pred_all_0),'\t','\t', precision_score(y_test, pred_all_0), "\t\tpredict all zero")
#     print(recall_score(y_test, pred_all_1),'\t','\t', precision_score(y_test, pred_all_1), "predict all one")
#     print(recall_score(y_test, pred_50_50),'\t', precision_score(y_test, pred_50_50), "predict 50-50")
#     print(recall_score(y_test, pred_90_10), precision_score(y_test, pred_90_10), "predict 90-10")

In [12]:
def display_importances_trees():
    # show feature importances
    feature_df = pd.DataFrame([X.columns, model.feature_importances_]).T
    feature_df.columns = ['feature','value']
    return feature_df.sort_values('value', ascending=False)
             
def display_metrics():
    print("\nMETRICS")
    print("Model recall: {}".format(recall_score(y_test, y_pred)))
    print("Model precision: {}".format(precision_score(y_test, y_pred)))
    print("Model accuracy: {}".format(model.score(X_test, y_test)))

    print ("\nCONFUSION MATRIX")
    print (confusion_matrix(y_test, y_pred))
    print ("\nkey:")
    print (" TN   FP ")
    print (" FN   TP ")

    print("\nRECALL AND ACCURACY FOR DIFFERNET MODELS")
    print("recall     \t precision   \tmodel")
    print(recall_score(y_test, y_pred), '\t',precision_score(y_test, y_pred), "my model")
    print(recall_score(y_test, pred_all_0),'\t','\t', precision_score(y_test, pred_all_0), "\t\tpredict all zero")
    print(recall_score(y_test, pred_all_1),'\t','\t', precision_score(y_test, pred_all_1), "predict all one")
    print(recall_score(y_test, pred_50_50),'\t', precision_score(y_test, pred_50_50), "predict 50-50")
    print(recall_score(y_test, pred_90_10), precision_score(y_test, pred_90_10), "predict 90-10")

# RANDOM FOREST CLASSIFIER

In [2]:
# model = RandomForestClassifier(max_depth=50, max_features=None, min_samples_leaf=3, n_estimators=100, n_jobs=-1)
# do_stuff(model)

model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# show metrics
display_metrics()

# show importances
display_importances_trees().head(5)


NameError: name 'RandomForestClassifier' is not defined

# GRADIENT BOOSTING CLASSIFIER

In [13]:
model = GradientBoostingClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# show metrics
display_metrics()

# show importances
display_importances_trees().head(5)


METRICS
Model recall: 0.9258202567760342
Model precision: 0.47046031170714026
Model accuracy: 0.7567987567987567

CONFUSION MATRIX
[[3572 1461]
 [ 104 1298]]

key:
 TN   FP 
 FN   TP 

RECALL AND ACCURACY FOR DIFFERNET MODELS
recall     	 precision   	model
0.925820256776 	 0.470460311707 my model
0.0 	 	 0.0 		predict all zero
1.0 	 	 0.217871017871 predict all one
0.502139800285 	 0.223421136147 predict 50-50
0.0984308131241 0.205970149254 predict 90-10


/Users/gandalf/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


,feature,value
9,created,0.343241
2,activeAt,0.126648
5,available,0.0686026
30,neighborhoods,0.0384192
29,minCost,0.0366821


# DECISION TREE CLASSIFIER

In [80]:
# fit model
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# show metrics
display_metrics()

# show importances
display_importances_trees().head(5)


METRICS
Model recall: 0.48288159771754635
Model precision: 0.498894620486367
Model accuracy: 0.7816627816627817

CONFUSION MATRIX
[[4353  680]
 [ 725  677]]

key:
 TN   FP 
 FN   TP 

RECALL AND ACCURACY FOR DIFFERNET MODELS
recall     	 precision   	model
0.482881597718 	 0.498894620486 my model
0.0 	 	 0.0 		predict all zero
1.0 	 	 0.217871017871 predict all one
0.48002853067 	 0.214126630608 predict 50-50
0.108416547789 0.223858615611 predict 90-10


/Users/gandalf/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


,feature,value
9,created,0.45375
2,activeAt,0.0529474
27,location,0.0490401
42,updated,0.036997
28,maxCost,0.0354758


# LOGISTIC REGRESSION

In [ ]:
model = LogisticRegression()
do_stuff(model)

feature_coefficients = np.argsort(model.coef_)
top_n = 10
# print("\nFEATURE RANKINGS")
# for n in range(top_n):
#     print(n+1, '\t',X.columns[feature_coefficients[-n-1]], '\t',sorted(model.feature_importances_)[-n-1])
        

In [ ]:
results_df = pd.DataFrame([X.columns,model.coef_[0]]).T
results_df.columns = ['feature','coefficients']
results_df['abs_val'] = results_df.coefficients.apply(abs)
results_df['sign'] = results_df.coefficients.apply(lambda s: s>0)

In [ ]:
results_df.head().sort_values('abs_val', ascending=False)

In [ ]:
def do_grid_search(X, y):
    '''
    X as 2d numpy array
    y as 1d numpy array
    
    PARAMETERS
    n_estimators: The number of trees in the forest
    criterion: gini or entropy
    max_features: The number of features to consider when looking for the best split
        If int, then consider max_features features at each split.
        If float, then max_features is a percentage and int(max_features * n_features) features are considered at each split.
        If “auto”, then max_features=sqrt(n_features).
        If “sqrt”, then max_features=sqrt(n_features) (same as “auto”).
        If “log2”, then max_features=log2(n_features).
        If None, then max_features=n_features.
    max_depth: The maximum depth of the tree
    n_jobs: The number of jobs to run in parallel for both fit and predict. If -1, then the number of jobs is set to the number of cores.
    '''
    
    # Split it up into our training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    # resample
    X_train, y_train = ms.oversample(X_train.as_matrix(), y_train.as_matrix(), .5)
    
    # Initalize our model here
    model = RandomForestClassifier()

    # Here are the params we are tuning
    param_grid = {'max_features' : [None],
                  'n_estimators' : [50,100,1000],
                  'max_depth': [50],
                  'min_samples_leaf': [3]
                  }

    # Plug in our model, params dict, and the number of jobs, then .fit()
    gs_cv = GridSearchCV(model, param_grid, n_jobs=-1).fit(X_train, y_train)

    # return the best score and the best params
    return gs_cv.best_score_, gs_cv.best_params_

In [ ]:
%%time
model = RandomForestClassifier(max_depth=50, max_features=None, min_samples_leaf=3, n_estimators=100, n_jobs=-1)
fit_model(model, X, y)

In [ ]:
%%time
n = do_grid_search(X, y)
print(n)

In [ ]:
%%time
model = LogisticRegression()
fit_model(model, X, y)

In [ ]:

print ("    Decision Tree:       ", get_scores(DecisionTreeClassifier, X_train, X_test, y_train, y_test))
print ("    Naive Bayes:         ", get_scores(MultinomialNB, X_train, X_test, y_train, y_test))

In [ ]:
# Calculate the standard deviation for feature importances across all trees

n = len(X.columns)

#importances = forest_fit.feature_importances_[:n]
importances = model.feature_importances_[:n]
std = np.std([tree.feature_importances_ for tree in model.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]
features = list(X.columns[indices])

# print("Feature ranking:")
# for f in range(n):
#     print("%d. %s (%f)" % (f + 1, features[f], importances[indices[f]]))

# Plot the feature importances of the forest
plt.figure(figsize=(16,12))
plt.title("Feature importances")
plt.bar(range(n), importances[indices], yerr=std[indices], color="r", align="edge", width = -.9)
plt.xticks(range(n), features, rotation=-75)
plt.xlim([-1, n])
plt.show()

In [ ]:
# Try modifying the number of trees, graph results
num_trees = range(5, 50, 5)
accuracies = []
for n in num_trees:
    tot = 0
    for i in range(5):
        rf = RandomForestClassifier(n_estimators=n)
        rf.fit(X_train, y_train)
        tot += rf.score(X_test, y_test)
    accuracies.append(tot / 5)
plt.plot(num_trees, accuracies)
plt.xlabel="num_trees"
plt.ylabel="accuracy"
plt.show()

In [ ]:
# Modifying the max features parameter
for nn in range(10):
    num_features = range(2, len(X.columns))
    accuracies = []
    for n in num_features:
        tot = 0
        for i in range(5):
            rf = RandomForestClassifier(max_features=n)
            rf.fit(X_train, y_train)
            tot += rf.score(X_test, y_test)
        accuracies.append(tot / 5)
    plt.plot(num_features, accuracies)
    plt.xlabel="num_features"
    plt.ylabel="accuracy"
plt.show()

In [ ]:
# Run all the other classifiers that we have learned so far in class
def get_scores(classifier, X_train, X_test, y_train, y_test, **kwargs):
    model = classifier(**kwargs)
    model.fit(X_train, y_train)
    y_predict = model.predict(X_test)
    return model.score(X_test, y_test), \
           precision_score(y_test, y_predict), \
           recall_score(y_test, y_predict)

print ("    Model,                Accuracy, Precision, Recall")
print ("    Random Forest:       ", get_scores(RandomForestClassifier, X_train, X_test, y_train, y_test, n_estimators=25, max_features=5))
print ("    Logistic Regression: ", get_scores(LogisticRegression, X_train, X_test, y_train, y_test))
print ("    Decision Tree:       ", get_scores(DecisionTreeClassifier, X_train, X_test, y_train, y_test))
print ("    Naive Bayes:         ", get_scores(MultinomialNB, X_train, X_test, y_train, y_test))

In [ ]:
def plot_roc(X, y, clf_class, title, **kwargs):
# def plot_roc(X, y, clf_class, kwargs):
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    kf = KFold(len(y), n_folds=5, shuffle=True)
    y_prob = np.zeros((len(y),2))
    mean_tpr = 0.0
    mean_fpr = np.linspace(0, 1, 100)
    all_tpr = []
    for i, (train_index, test_index) in enumerate(kf):
        X_train, X_test = X[train_index], X[test_index]
        y_train = y[train_index]
        clf = clf_class(**kwargs)
        clf.fit(X_train,y_train)
        # Predict probabilities, not classes
        y_prob[test_index] = clf.predict_proba(X_test)
        fpr, tpr, thresholds = roc_curve(y[test_index], y_prob[test_index, 1])
        mean_tpr += interp(mean_fpr, fpr, tpr)
        mean_tpr[0] = 0.0
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, lw=1, label='ROC fold %d (area = %0.2f)' % (i, roc_auc))
    mean_tpr /= len(kf)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    plt.plot(mean_fpr, mean_tpr, 'k--',label='Mean ROC (area = %0.2f)' % mean_auc, lw=2)

    plt.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6), label='Random')
    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
#     plt.xlabel('False Positive Rate')
#     plt.ylabel('True Positive Rate')
    plt.title(title + 'ROC')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
xfake, yfake = np.random.rand(100,6), np.random.randint(0,2, size=(100,))
plt.figure(figsize=(20,16))
print ("Visualize the roc curve of each model")
plot_roc(xfake, yfake, RandomForestClassifier, 'Random_Forest', n_estimators=25, max_features=5)
#plot_roc(X, y, LogisticRegression, 'Logistic_Regrssion')
#plot_roc(X, y, DecisionTreeClassifier, 'Decision_Tree')
#plot_roc(X, y, MultinomialNB, 'Naive_Bayes') error
print('\nPlotting completed.')

In [ ]:
print ("Visualize the roc curve of each model")
plot_roc(X, y, RandomForestClassifier, 'Random_Forest', n_estimators=25, max_features=5)
plot_roc(X, y, LogisticRegression, 'Logistic_Regrssion')
plot_roc(X, y, DecisionTreeClassifier, 'Decision_Tree')
#plot_roc(X, y, MultinomialNB, 'Naive_Bayes') error
print('\nPlotting completed.')